# Application AutoEncoder for SIR Model estimation

In [ ]:
#libraries
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, MaxPool2D, Add
from tensorflow.keras import Model
import tensorflow.keras as keras
from keras import layers
import tensorflow as tf

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from keras import regularizers

In [ ]:
# split input_data
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from  sklearn.preprocessing import StandardScaler

In [ ]:
# Import data 
input_data=pd.read_csv("C:/Users/pc/Desktop/PhD/AI Phd/AutoEncoder/Input.csv",delimiter=";",sep=",")
output_data=pd.read_csv("C:/Users/pc/Desktop/PhD/AI Phd/AutoEncoder/Output.csv",delimiter=";",sep=",")

In [ ]:
input_data=input_data.astype('float32')
output_data=output_data.astype('float32')

In [ ]:
input_data.shape

In [ ]:
X=StandardScaler().fit_transform(input_data)
Y=StandardScaler().fit_transform(output_data)

In [ ]:
x_train, x_test = train_test_split(X,test_size = 0.2, random_state = 2)
y_train, y_test = train_test_split(Y,test_size = 0.2, random_state = 2)

In [ ]:
x_train

In [ ]:
# Encoder SIR 

latent_dim = 2

encoder_inputs =  keras.Input(shape=(3,))
x = layers.Dense(10, activation='linear')(encoder_inputs)
x = layers.Dense(5, activation='linear')(x)

z = layers.Dense(latent_dim, activation='linear',name="z")(x)

encoder = keras.Model(encoder_inputs, z, name="encoder")
encoder.summary()

In [ ]:
# Decoder SIR

latent_inputs = keras.Input(shape=(latent_dim,))
x = layers.Dense(5, activation='linear')(latent_inputs)
x = layers.Dense(10, activation='linear')(x)
decoder_outputs = layers.Dense(3, activation='linear')(x)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

In [ ]:
class AE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(AE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.odeint_tracker = keras.metrics.Mean(name="odeint")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.odeint_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, reconstruction), axis=1
                )
            
           
            x_tilde= data + tf.reduce_sum(data)
            odeint = tf.reduce_sum(
                    keras.losses.binary_crossentropy(data, x_tilde), axis=1
                )
           
            
            
            total_loss = reconstruction_loss + odeint
            
            
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.odeint_tracker.update_state(odeint)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "odeint": self.odeint_tracker.result(),
        }

In [ ]:
autoencoder = AE(encoder, decoder)

In [ ]:
autoencoder.compile(optimizer=keras.optimizers.Adam())

In [ ]:
#Training
hist=autoencoder.fit(x_train, y_train, epochs=3, batch_size=128)

In [ ]:
# summarize history for loss
plt.plot(hist.history['loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()